# Proyecto database generator Mónica Serrano

In [4]:
import pandas as pd
import numpy as np
import mysql.connector
pd.options.display.max_columns=None

## Cagamos el fichero, lo analizamos y lo preparamos.

In [23]:
villager = pd.read_csv('files/villagers.csv')
villager.head()

,Name,Species,Gender,Personality,Hobby,Birthday,Catchphrase,Favorite Song,Style 1,Style 2,Color 1,Color 2,Wallpaper,Flooring,Furniture List,Filename,Unique Entry ID
0,Admiral,Bird,Male,Cranky,Nature,27-Jan,aye aye,Steep Hill,Cool,Cool,Black,Blue,dirt-clod wall,tatami,717;1849;7047;2736;787;5970;3449;3622;3802;410...,brd06,B3RyfNEqwGmcccRC3
1,Agent S,Squirrel,Female,Peppy,Fitness,2-Jul,sidekick,Go K.K. Rider,Active,Simple,Blue,Black,concrete wall,colorful tile flooring,7845;7150;3468;4080;290;3971;3449;1708;4756;25...,squ05,SGMdki6dzpDZyXAw5
2,Agnes,Pig,Female,Big Sister,Play,21-Apr,snuffle,K.K. House,Simple,Elegant,Pink,White,gray molded-panel wall,arabesque flooring,4129;7236;7235;7802;896;3428;4027;7325;3958;71...,pig17,jzWCiDPm9MqtCfecP
3,Al,Gorilla,Male,Lazy,Fitness,18-Oct,ayyyeee,Go K.K. Rider,Active,Active,Red,White,concrete wall,green rubber flooring,1452;4078;4013;833;4116;3697;7845;3307;3946;39...,gor08,LBifxETQJGEaLhBjC
4,Alfonso,Alligator,Male,Lazy,Play,9-Jun,it'sa me,Forest Life,Simple,Simple,Red,Blue,yellow playroom wall,green honeycomb tile,4763;3205;3701;1557;3623;85;3208;3584;4761;121...,crd00,REpd8KxB8p9aGBRSE


In [24]:
villager.columns

Index(['Name', 'Species', 'Gender', 'Personality', 'Hobby', 'Birthday',
       'Catchphrase', 'Favorite Song', 'Style 1', 'Style 2', 'Color 1',
       'Color 2', 'Wallpaper', 'Flooring', 'Furniture List', 'Filename',
       'Unique Entry ID'],
      dtype='object')

In [25]:
columnas=['Name', 'Species', 'Gender', 'Personality', 'Hobby', 'Birthday']
df_sql = villager[columnas]

In [26]:
def cambio_columnas(dataframe):
    """
   Esta función cambiar el nombre de las columnas a minúscula.
  Args: 
       dataframe(dataframe): el dataframe el cual vamos a cambiar sus columnas 
   Returns: aunque no tiene return, esta función modifica el mismo dataframe con todas las columnas en minúsculas.
   """
    columnas_nuevas = {col :  col.lower() for col in dataframe.columns}
    dataframe.rename(columns = columnas_nuevas, inplace = True)

In [27]:
cambio_columnas(df_sql)

/var/folders/cf/7kf7s8j17wn53znl0ywm912w0000gp/T/ipykernel_6431/1161268461.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe.rename(columns = columnas_nuevas, inplace = True)


In [28]:
def minus(elemento):
    """
   Esta función cambiar el nombre de los elementos de una columna a minúscula.
  Args: 
       elemento(str): el elemento que queremos modificar.
   Returns: devuelve el elemento en minúscula
   """
    return elemento.lower()

NOTA IMPORTANTE: En el caso de que la columna tenga nulos esta función puede dar error, por lo que se deberá utilizar la siguiente:

In [ ]:
def minus_nulos(elemento):
    """
   Esta función cambiar el nombre de los elementos de una columna a minúscula.
  Args: 
       elemento(str): el elemento que queremos modificar.
   Returns: devuelve el elemento en minúscula
   """
    try:
        return elemento.lower()
    except:
        return np.nan

In [29]:
df_sql['name_index'] = df_sql['name'].apply(minus)

/var/folders/cf/7kf7s8j17wn53znl0ywm912w0000gp/T/ipykernel_6431/1044701161.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sql['name_index'] = df_sql['name'].apply(minus)


In [30]:
df_sql = df_sql.set_index('name_index')

In [31]:
df_prueba = df_sql.reset_index().reset_index().set_index('name_index')
df_prueba = df_prueba[['name' ,'species','gender']]

In [32]:
df_prueba

,name,species,gender
name_index,,,
admiral,Admiral,Bird,Male
agent s,Agent S,Squirrel,Female
agnes,Agnes,Pig,Female
al,Al,Gorilla,Male
alfonso,Alfonso,Alligator,Male
...,...,...,...
winnie,Winnie,Horse,Female
wolfgang,Wolfgang,Wolf,Male
yuka,Yuka,Koala,Female


## Creamos las funciones que compondrán nuestra clase.

In [66]:
def crear_base_datos(base_datos='base_datos_1',valor_user='root', valor_password='AlumnaAdalab',valor_host='127.0.0.1'):
    """
   Esta función crea una base de datos de forma automatica.
  Args: 
       base_datos(str): el nombre de la base de datos qeu se va a crear
       valor_user(str): el nombre del usuario de MySQL.
       valor_password(str): contraseña correspondiente al usuario de MySQL.
       valor_host(str): el valor del host.
   Returns: a pesar de no tener return, crea una base de datos en el servidor de MySQL proporcionado.
   """
    atencion = input(f'Vas a crear la base de datos {base_datos},¿Quieres cambiar el nombre? (si/no)').lower()
    if atencion == 'si':
        eleccion = input('¿Cómo quieres que se llame tu base de datos?')
        base_datos = eleccion.lower()


    cnx = mysql.connector.connect(user=valor_user, password = valor_password,host = valor_host)

    mycursor = cnx.cursor()

    try:

        mycursor.execute(f"CREATE DATABASE {base_datos}")
        print(f'Se ha creado la base de datos {base_datos}')
        print(f'Estas operando con la base de datos {base_datos}')
    except mysql.connector.Error as err:

        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

    opcion = input('¿Quieres crear otra base de datos?(si/no)')

    if opcion.lower() == 'si':
        nuevo_nombre = input('¿Cómo quieres que se llame la nueva base de datos?').lower()
        base_datos = nuevo_nombre
        crear_base_datos()

In [67]:
def crear_tabla(tabla='tabla1',base_datos='base_datos_1',valor_user='root', valor_password='AlumnaAdalab',valor_host='127.0.0.1'):
    """
   Esta función crea una tabla de forma automatica.
  Args: 
       tabla(str): el nombre de la tabla que se va a crear
       base_datos(str): el nombre de la base de datos donde se va a crear la tabla
       valor_user(str): el nombre del usuario de MySQL.
       valor_password(str): contraseña correspondiente al usuario de MySQL.
       valor_host(str): el valor del host.
   Returns: a pesar de no tener return, crea una tabla en una base de datos del servidor de MySQL proporcionado.
   """
    nombre_columnas = input('Indica el nombre de las columnas separados por comas')
    tipo_columnas = input('Indica el tipo de dato de cada columna separados por comas')
    nulo = input('Indica si puede ser nulo o no')
    caracteres = input('¿Cuantos caracteres quieres que tengan los de tipo VARCHAR')
    nombre_columnas = nombre_columnas.split(',')
    tipo_columnas = tipo_columnas.split(',')
    nulo = nulo.split(',')
    nombre_columnas = [elemento.strip().lower() for elemento in nombre_columnas]
    tipo_columnas = [elemento.strip().upper() for elemento in tipo_columnas]
    nulo = [elemento.strip().lower() for elemento in nulo]
    if len(nombre_columnas) == len(tipo_columnas) == len(nulo):
        query = f"""CREATE TABLE IF NOT EXISTS {tabla} ("""
        for indice in range(len(nombre_columnas)):
            if tipo_columnas[indice] in ['INT','VARCHAR','FLOAT']:
                if indice == len(nombre_columnas) -1:
                    if tipo_columnas[indice] == 'VARCHAR':
                        if nulo[indice].lower() == 'si':
                            query = query + ' ' + nombre_columnas[indice] + ' ' + tipo_columnas[indice] + '(' + caracteres + ')' + ')'
                        elif nulo[indice].lower() == 'no':
                            query = query + ' ' + nombre_columnas[indice] + ' ' + tipo_columnas[indice] +'(' + caracteres + ')' + ' ' + 'NOT NULL' +')'
                    else:
                        if nulo[indice].lower() == 'si':
                            query = query + ' ' + nombre_columnas[indice] + ' ' + tipo_columnas[indice] + ')'
                        elif nulo[indice].lower() == 'no':
                            query = query + ' ' + nombre_columnas[indice] + ' ' + tipo_columnas[indice] + ' ' + 'NOT NULL' +')'
                else:
                    if tipo_columnas[indice] == 'VARCHAR':
                        if nulo[indice].lower() == 'si':
                            query = query + ' ' + nombre_columnas[indice] + ' ' + tipo_columnas[indice] + '(' + caracteres + ')' + ','
                        elif nulo[indice].lower() == 'no':
                            query = query + ' ' + nombre_columnas[indice] + ' ' + tipo_columnas[indice] + '(' + caracteres + ')' + ' ' + 'NOT NULL' + ','
                    else:
                        if nulo[indice].lower() == 'si':
                            query = query + ' ' + nombre_columnas[indice] + ' ' + tipo_columnas[indice] + ','
                        elif nulo[indice].lower() == 'no':
                            query = query + ' ' + nombre_columnas[indice] + ' ' + tipo_columnas[indice] + ' ' + 'NOT NULL' + ','
    else:
        print('No has especificado el mismo numero de elementos en las tres listas')
    cnx = mysql.connector.connect(user = valor_user, password = valor_password, host = valor_host, database = base_datos)
    mycursor = cnx.cursor()
    try: 
        mycursor.execute(query)
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

In [68]:
def conocer_columnas_tabla(tabla='tabla1',base_datos='base_datos_1',valor_user='root', valor_password='AlumnaAdalab',valor_host='127.0.0.1'):
    """
   Esta función extrae las columnas que hay en una tabla concreta.
  Args: 
       tabla(str): la tabla de donde se van a extraer las columnas.
       base_datos(str): el nombre de la base de datos donde se encuentra la tabla.
       valor_user(str): el nombre del usuario de MySQL.
       valor_password(str): contraseña correspondiente al usuario de MySQL.
       valor_host(str): el valor del host.
   Returns: devuelve una lista con todas las columnas que tiene la tabla proporcionada.
   """
    cnx = mysql.connector.connect(user = valor_user, password = valor_password, host = valor_host, database = base_datos)
    mycursor = cnx.cursor()
    try: 
        mycursor.execute(f'SHOW COLUMNS FROM {tabla}')
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)
    resultado = [elemento[0] for elemento in mycursor]
    return resultado

In [20]:
def tipo_dato(indice,tabla='tabla1',base_datos='base_datos_prueba',valor_user='root', valor_password='AlumnaAdalab',valor_host='127.0.0.1'):
    """
   Esta función extrae el tipo de dato de una columna.
  Args:
       indice(int): el número de indice donde se encuentra el tipo de dato de la columa especificada.
       tabla(str): la tabla de donde se encuentra la columna.
       base_datos(str): el nombre de la base de datos donde se encuentra la tabla.
       valor_user(str): el nombre del usuario de MySQL.
       valor_password(str): contraseña correspondiente al usuario de MySQL.
       valor_host(str): el valor del host.
   Returns: devuelve una lista con todas las columnas que tiene la tabla proporcionada.
   """
    cnx = mysql.connector.connect(database=base_datos,user=valor_user, password=valor_password,host=valor_host)
    mycursor = cnx.cursor()
    try: 
        mycursor.execute(f'SHOW COLUMNS FROM {tabla}')
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)
    resultado = [elemento[1] for elemento in mycursor]
    return resultado[indice]


In [69]:
def comprobar_columnas(lista_columnas=[],tabla='tabla1',base_datos='base_datos_1'):
    """
   Esta función comprueba que las columnas proporcionadas están una tabla concreta.
  Args: 
       lista columnas (lista): lista donde están los nombres de las columnas propuestas por el usuario.
       tabla(str): la tabla de donde se van a comprobar las columnas.
       base_datos(str): el nombre de la base de datos donde se encuentra la tabla.
   Returns: devuelve un booleano dependiendo si las columnas están presentes en la tabla o no.
   """
    resultado_columnas = conocer_columnas_tabla(tabla,base_datos)

    for indice in range(len(lista_columnas)):

        if indice == len(lista_columnas)-1:

            if lista_columnas[indice] in resultado_columnas:
                return True
        else:

            if lista_columnas[indice] not in resultado_columnas:
                return False

In [70]:
def clave_primaria(tabla='tabla1',base_datos='base_datos_1',valor_user='root', valor_password='AlumnaAdalab',valor_host='127.0.0.1'):
    """
   Esta función establece la primary key de una tabla.
  Args: 
       tabla(str): la tabla de donde se va a establecer la PK.
       base_datos(str): el nombre de la base de datos donde se encuentra la tabla.
       valor_user(str): el nombre del usuario de MySQL.
       valor_password(str): contraseña correspondiente al usuario de MySQL.
       valor_host(str): el valor del host.
   Returns: aunque no tiene return, establece la PK de una tabla.
   """
    resultado = conocer_columnas_tabla(tabla,base_datos)
    columna = input(f'De estas columnas, {resultado} ¿Cuál quieres que sea PK?')
    nombre_cons = input('¿Como quieres que se llame la constraint?')
    if columna.lower() in resultado:
        cnx = mysql.connector.connect(user = valor_user, password = valor_password, host = valor_host, database = base_datos)
        mycursor = cnx.cursor()
        query= f'ALTER TABLE {tabla} ADD CONSTRAINT {nombre_cons} PRIMARY KEY ({columna});'
        try: 
            mycursor.execute(query)
        except mysql.connector.Error as err:
            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg)

In [71]:
def modificar_columna(tabla='tabla1',base_datos='base_datos_1',valor_user='root', valor_password='AlumnaAdalab',valor_host='127.0.0.1'):
    """
   Esta función modifica el tipo de dato, si es nulo o si es autoincrementable una columna de una tabla.
  Args: 
       tabla(str): la tabla de donde se va a cambiar la columna.
       base_datos(str): el nombre de la base de datos donde se encuentra la tabla.
       valor_user(str): el nombre del usuario de MySQL.
       valor_password(str): contraseña correspondiente al usuario de MySQL.
       valor_host(str): el valor del host.
   Returns: aunque no tiene return, modifica la columna especificada.
   """
    resultado = conocer_columnas_tabla(tabla,base_datos)
    columna = input(f'De estas columnas, {resultado} ¿Cuál quieres que sea modificada?').lower()
    tipo_dato = input('¿Como quieres que sea el tipo de dato?').upper()
    auto = input('¿Quiers que sea auto-incrementable?')
    if tipo_dato == 'VARCHAR':
        caracteres = input('Especifica el numero de caracteres')
    nulo = input('¿Quieres que pueda ser nulo').lower()
    if columna in resultado:
        cnx = mysql.connector.connect(user = valor_user, password = valor_password, host = valor_host, database = base_datos)
        mycursor = cnx.cursor()
        query= f'ALTER TABLE {tabla} MODIFY COLUMN  {columna}'
        if tipo_dato in ['INT','VARCHAR','FLOAT']:
            if tipo_dato == 'VARCHAR':
                if nulo == 'si':
                    query = query + ' ' + tipo_dato + '('+ caracteres + ')'
                elif nulo == 'no':
                    query = query + ' ' + tipo_dato + '('+ caracteres + ')' + ' '+'NOT NULL'
            else:
                if auto.lower() == 'si':
                    if nulo == 'si':
                        query = query + ' ' + tipo_dato
                    elif nulo == 'no':
                        query = query + ' ' + tipo_dato + ' ' + 'NOT NULL' + ' ' + 'AUTO_INCREMENT'
                else:
                    if nulo == 'si':
                        query = query + ' ' + tipo_dato 
                    elif nulo == 'no':
                        query = query + ' ' + tipo_dato + ' ' + 'NOT NULL'
        try: 
            mycursor.execute(query)
        except mysql.connector.Error as err:
            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg)

In [72]:
def conocer_tablas(base_datos='base_datos_1',valor_user='root', valor_password='AlumnaAdalab',valor_host='127.0.0.1'):
    """
   Esta función extrae las tablas de una base de datos.
  Args: 
       base_datos(str): el nombre de la base de datos donde se encuentran las tablas.
       valor_user(str): el nombre del usuario de MySQL.
       valor_password(str): contraseña correspondiente al usuario de MySQL.
       valor_host(str): el valor del host.
   Returns: devuelve una lista con las tablas de la base de datos especificada.
   """
    cnx = mysql.connector.connect(user=valor_user, password = valor_password,host = valor_host)

    mycursor = cnx.cursor()
    mycursor.execute("SHOW DATABASES")

    bases_de_datos = [database[0] for database in mycursor]

    if base_datos in bases_de_datos:

        cnx = mysql.connector.connect(database=base_datos,user=valor_user, password = valor_password,host = valor_host)

        mycursor = cnx.cursor()

        mycursor.execute("SHOW TABLES")

        conocer_tablas = [tablas[0] for tablas in mycursor]
        return conocer_tablas
       

In [73]:
def clave_foranea(tabla='tabla1',base_datos='base_datos_1',valor_user='root', valor_password='AlumnaAdalab',valor_host='127.0.0.1'):
    """
   Esta función establace la clave foranea de una tabla.
  Args: 
       tabla(str): la tabla de donde se va a establecer la FK.
       base_datos(str): el nombre de la base de datos donde se encuentra la tabla.
       valor_user(str): el nombre del usuario de MySQL.
       valor_password(str): contraseña correspondiente al usuario de MySQL.
       valor_host(str): el valor del host.
   Returns: aunque no tiene return, mestablace la FK de la tabla especificada.
   """
    resultado = conocer_columnas_tabla(tabla,base_datos)
    columna = input(f'De estas columnas, {resultado} ¿Cuál quieres que sea foreign key?').lower()
    if columna in resultado:
        tablas = conocer_tablas(base_datos)
        eleccion =input(f'De estas tablas, {tablas}¿Cuál tendrá relación con la columna?').lower()
        if eleccion in tablas:
            resultado2 = conocer_columnas_tabla(eleccion,base_datos)
            columna2 = input(f'De estas columnas, {resultado2} ¿Cuál quieres que tenga relación con la FK?').lower()
            if columna2 in resultado2:
                cnx = mysql.connector.connect(user = valor_user, password = valor_password, host = valor_host, database = base_datos)
                mycursor = cnx.cursor()
                query= f'ALTER TABLE {tabla} ADD FOREIGN KEY ({columna}) REFERENCES {eleccion}  ({columna2})'
                try: 
                    mycursor.execute(query)
                except mysql.connector.Error as err:
                    print(err)
                    print("Error Code:", err.errno)
                    print("SQLSTATE", err.sqlstate)
                    print("Message", err.msg)

In [74]:
def insertar_columna(tabla='tabla1',base_datos='base_datos_1',valor_user='root', valor_password='AlumnaAdalab',valor_host='127.0.0.1'):
    """
   Esta función introduce una nueva columna en una tabla.
  Args: 
       tabla(str): la tabla de donde se va a introducir la nueva columna.
       base_datos(str): el nombre de la base de datos donde se encuentra la tabla.
       valor_user(str): el nombre del usuario de MySQL.
       valor_password(str): contraseña correspondiente al usuario de MySQL.
       valor_host(str): el valor del host.
   Returns: aunque no tiene return, inserta una nueva columna en la tabla especificada.
   """
    columna = input(f'Escribe el nombre de la nueva columna').lower()
    tipo_dato = input('¿Como quieres que sea el tipo de dato?').upper()
    if tipo_dato == 'VARCHAR':
        caracteres = input('Especifica el numero de caracteres')
    nulo = input('¿Quieres que pueda ser nulo').lower()
  
    cnx = mysql.connector.connect(user = valor_user, password = valor_password, host = valor_host, database = base_datos)
    mycursor = cnx.cursor()
    query= f'ALTER TABLE {tabla} ADD COLUMN  {columna}'
    if tipo_dato in ['INT','VARCHAR','FLOAT']:
        if tipo_dato == 'VARCHAR':
            if nulo == 'si':
                query = query + ' ' + tipo_dato + '('+ caracteres + ')'
            elif nulo == 'no':
                query = query + ' ' + tipo_dato + '('+ caracteres + ')' + ' ' + 'NOT NULL'
        else:
            if nulo == 'si':
                query = query + ' ' + tipo_dato
            elif nulo == 'no':
                query = query + ' ' + tipo_dato + ' ' + 'NOT NULL'
    try: 
        mycursor.execute(query)
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

In [75]:
def insertar_datos(df=df_prueba,tabla='tabla1',base_datos='base_datos_1',valor_user='root', valor_password='AlumnaAdalab',valor_host='127.0.0.1'):
    """
   Esta función inserta datos en una tabla.
  Args: 
       df(dataframe): el dataframe que contiene los datos que queremos insertar en la tabla.
       tabla(str): la tabla de donde se van a insertar los datos.
       base_datos(str): el nombre de la base de datos donde se encuentra la tabla.
       valor_user(str): el nombre del usuario de MySQL.
       valor_password(str): contraseña correspondiente al usuario de MySQL.
       valor_host(str): el valor del host.
   Returns: aunque no tiene return, inserta todos los datos en la tabla especificada.
   """
    cnx = mysql.connector.connect(user=valor_user, password = valor_password,host = valor_host)

    mycursor = cnx.cursor()
    mycursor.execute("SHOW DATABASES")

    bases_de_datos = [database[0] for database in mycursor]

    if base_datos in bases_de_datos:

        resultado = conocer_columnas_tabla(tabla,base_datos)
            
        eleccion = input(f'De estas columnas {resultado}, ¿En cuales vas a introducir datos?')
        eleccion = eleccion.split(',')
        
        if comprobar_columnas(eleccion,tabla,base_datos):
            num_valores ='(%s'

            for i in range(len(eleccion)-1):
                num_valores = num_valores +',%s'
            num_valores = num_valores+')'
            
            codigo_sql=f'INSERT INTO {tabla} {tuple(eleccion)} VALUES {num_valores}'
            codigo_sql=codigo_sql.replace("'","")
            lista = []
            for indice, fila in df.iterrows():
                lista=[]
                for i in df.loc[indice,:].values:
                    lista.append(i)
                valores = tuple(lista)
                cnx = mysql.connector.connect(user = valor_user, password = valor_password, host = valor_host, database = base_datos)
                mycursor = cnx.cursor()
                                
                try: 
                    mycursor.execute(codigo_sql, valores)
                    cnx.commit()
                except mysql.connector.Error as err:
                    print(err)
                    print("Error Code:", err.errno)
                    print("SQLSTATE", err.sqlstate)
                    print("Message", err.msg)
        else:
            eleccion_2 = input('Las columnas introducidas no corresponde con la tabla, ¿Quieres volverlo a intentar? (si/no)').lower()
            if eleccion_2 == 'si':
                insertar_datos()
    else:
        eleccion_3 = input(f'No existe la base de datos {base_datos}, ¿Quieres crearla? (si/no)')
        if eleccion_3.lower() == 'si':
            crear_base_datos()    

In [76]:
def eliminar_registro(tabla='tabla1',base_datos='base_datos_1',valor_user='root', valor_password='AlumnaAdalab',valor_host='127.0.0.1'):
    """
   Esta función elimina todos los registros de la tabla que cumple una cierta condición.
  Args: 
       tabla(str): la tabla de donde se encuentran los registros que queremos eliminar.
       base_datos(str): el nombre de la base de datos donde se encuentra la tabla.
       valor_user(str): el nombre del usuario de MySQL.
       valor_password(str): contraseña correspondiente al usuario de MySQL.
       valor_host(str): el valor del host.
   Returns: aunque no tiene return, elimina los registros especificados.
   """
    cnx = mysql.connector.connect(user=valor_user, password = valor_password,host = valor_host)

    mycursor = cnx.cursor()
    mycursor.execute("SHOW DATABASES")

    bases_de_datos = [database[0] for database in mycursor]

    if base_datos in bases_de_datos:
        resultado = conocer_columnas_tabla(tabla,base_datos)
        columna = input(f'¿Qué columna tomamos como referencia para borrar el registro?{resultado}').lower()
        if columna in resultado:
            cnx = mysql.connector.connect(user = valor_user, password = valor_password, host = valor_host, database = base_datos)
            mycursor = cnx.cursor()
            query = f'SELECT DISTINCT {columna} FROM {tabla}'
            query=query.replace("'","")
            try: 
                mycursor.execute(query)
            except mysql.connector.Error as err:
                print(err)
                print("Error Code:", err.errno)
                print("SQLSTATE", err.sqlstate)
                print("Message", err.msg)

            elementos_unicos = [elemento[0] for elemento in mycursor]
            eleccion_elemento = input(f'Entre los registros disponibles: {elementos_unicos}, ¿Cuál quieres borrar?').lower().capitalize()

            if eleccion_elemento in elementos_unicos:

                codigo_sql=f"DELETE FROM {tabla} WHERE {columna} = '{eleccion_elemento}'" 
                cnx = mysql.connector.connect(user = valor_user, password = valor_password, host = valor_host, database = base_datos)
                mycursor = cnx.cursor()
                                    
                try: 
                    mycursor.execute(codigo_sql)
                    cnx.commit()
                    print('Se ha eliminado correctamente todos los registros con esa condición en la base de datos.')
                except mysql.connector.Error as err:
                    print(err)
                    print("Error Code:", err.errno)
                    print("SQLSTATE", err.sqlstate)
                    print("Message", err.msg)
        else:
            eleccion_3 = input(f'No existe ningún registro con esas caracteristicas, ¿Quieres probar de nuevo? (si/no)')
            if eleccion_3.lower() == 'si':
                eliminar_registro()
    else:
        eleccion_3 = input(f'No existe la base de datos {base_datos}, ¿Quieres crearla? (si/no)')
        if eleccion_3.lower() == 'si':
            crear_base_datos()

In [77]:
def consultar_registro(tabla='tabla1',base_datos='base_datos_1',valor_user='root', valor_password='AlumnaAdalab',valor_host='127.0.0.1'):
    """
   Esta función consulta todos los registros de la tabla que cumple una cierta condición.
  Args: 
       tabla(str): la tabla de donde se encuentran los registros que queremos consultar.
       base_datos(str): el nombre de la base de datos donde se encuentra la tabla.
       valor_user(str): el nombre del usuario de MySQL.
       valor_password(str): contraseña correspondiente al usuario de MySQL.
       valor_host(str): el valor del host.
   Returns: aunque no tiene return, consulta los registros especificados.
   """
    cnx = mysql.connector.connect(user=valor_user, password = valor_password,host = valor_host)

    mycursor = cnx.cursor()
    mycursor.execute("SHOW DATABASES")

    bases_de_datos = [database[0] for database in mycursor]

    if base_datos in bases_de_datos:
        resultado = conocer_columnas_tabla(tabla,base_datos)
        columna = input(f'¿Qué columna tomamos como referencia para consultar el registro?{resultado}').lower()
        if columna in resultado:
            cnx = mysql.connector.connect(user = valor_user, password = valor_password, host = valor_host, database = base_datos)
            mycursor = cnx.cursor()
            query = f'SELECT DISTINCT {columna} FROM {tabla}'
            query=query.replace("'","")
            try: 
                mycursor.execute(query)
            except mysql.connector.Error as err:
                print(err)
                print("Error Code:", err.errno)
                print("SQLSTATE", err.sqlstate)
                print("Message", err.msg)

            elementos_unicos = [elemento[0] for elemento in mycursor]
            eleccion_elemento = input(f'Entre los registros disponibles: {elementos_unicos}, ¿Cuál quieres consultar?').lower().capitalize()

            if eleccion_elemento in elementos_unicos:

                codigo_sql=f"SELECT * FROM {tabla} WHERE {columna} = '{eleccion_elemento}'" 
                cnx = mysql.connector.connect(user = valor_user, password = valor_password, host = valor_host, database = base_datos)
                mycursor = cnx.cursor()
                                    
                try: 
                    mycursor.execute(codigo_sql)
                    registros = [elemento for elemento in mycursor]
                    print(f'Los registros concordantes son: {registros}')
                    return registros
                except mysql.connector.Error as err:
                    print(err)
                    print("Error Code:", err.errno)
                    print("SQLSTATE", err.sqlstate)
                    print("Message", err.msg)
        else:
            eleccion_3 = input(f'No existe ningún registro con esas caracteristicas, ¿Quieres probar de nuevo? (si/no)')
            if eleccion_3.lower() == 'si':
                consultar_registro()
    else:
        eleccion_3 = input(f'No existe la base de datos {base_datos}, ¿Quieres crearla? (si/no)')
        if eleccion_3.lower() == 'si':
            crear_base_datos()

In [ ]:
def cambiar_base_datos(base_datos='base_datos_1',valor_user='root', valor_password='AlumnaAdalab',valor_host='127.0.0.1'):
    """
   Esta función cambia de base de datos.
  Args: 
       base_datos(str): el nombre de la base de datos con la que se esta operando en ese momento.
       valor_user(str): el nombre del usuario de MySQL.
       valor_password(str): contraseña correspondiente al usuario de MySQL.
       valor_host(str): el valor del host.
   Returns: aunque no tiene return, cambia de base de datos.
   """
    cnx = mysql.connector.connect(user=valor_user, password = valor_password,host = valor_host)

    mycursor = cnx.cursor()
    mycursor.execute("SHOW DATABASES")

    bases_de_datos = [database[0] for database in mycursor]

    opcion = input(f'Estas operando actualmente con la base de datos {base_datos},¿Quieres continuar?(si/no)')

    if opcion.lower() == 'si':
            
        print(f'Las bases de datos disponibles son {bases_de_datos}')

        eleccion = input('Elige la base de datos con la quieres operar')

        if eleccion.lower() in bases_de_datos:

            base_datos = eleccion.lower()
            print(f'Estas operando con la base de datos {base_datos}')

        else:

            eleccion_2 = input('Esa eleccion no es correcta, ¿Quieres probar de nuevo? (si/no)')

            if eleccion_2.lower() == 'si':

                cambiar_base_datos()

            else:

                print(f'La base de datos con la que estás operando es {base_datos}')
    else:
        print(f'La base de datos con la que estás operando es {base_datos}')

In [ ]:
def cambiar_tabla(tabla='tabla1',base_datos='base_datos_1',valor_user='root', valor_password='AlumnaAdalab',valor_host='127.0.0.1'):
    """
   Esta función cambia de tabla.
  Args: 
       tabla(str): el nombre de la tabla con la que se esta operando actualmente.
       base_datos(str): el nombre de la base de datos donde se encuentra la tabla.
       valor_user(str): el nombre del usuario de MySQL.
       valor_password(str): contraseña correspondiente al usuario de MySQL.
       valor_host(str): el valor del host.
   Returns: aunque no tiene return, cambia de tabla.
   """
    cnx = mysql.connector.connect(user=valor_user, password = valor_password,host = valor_host)

    mycursor = cnx.cursor()
    mycursor.execute("SHOW DATABASES")

    bases_de_datos = [database[0] for database in mycursor]

    if base_datos in bases_de_datos:

        cnx = mysql.connector.connect(database=base_datos,user=valor_user, password = valor_password,host = valor_host)

        mycursor = cnx.cursor()

        mycursor.execute("SHOW TABLES")

        conocer_tablas = [tablas[0] for tablas in mycursor]
        print(f'Las tablas disponibles son {conocer_tablas}')

        eleccion = input('Elige la tabla con la quieres operar')

        if eleccion.lower() in conocer_tablas:

            tabla = eleccion.lower()
            print(f'Estas operando con la base de datos {eleccion.lower()}')

        else:

            eleccion_2 = input('Esa eleccion no es correcta, ¿Quieres probar de nuevo? (si/no)')

            if eleccion_2.lower() == 'si':

                cambiar_tabla()

            else:

                print(f'La base de datos con la que estás operando es {tabla}')
    else:

        eleccion_3 = input(f'No existe la base de datos {base_datos}, ¿Quieres crearla? (si/no)')
        if eleccion_3.lower() == 'si':
            crear_base_datos()

In [ ]:
def eliminar_columna(tabla='tabla1',base_datos='base_datos_1',valor_user='root', valor_password='AlumnaAdalab',valor_host='127.0.0.1'):
    """
   Esta función elimina una columna.
  Args: 
       tabla(str): el nombre de la tabla en la que se encuentra la columna.
       base_datos(str): el nombre de la base de datos con la que se esta operando en ese momento.
       valor_user(str): el nombre del usuario de MySQL.
       valor_password(str): contraseña correspondiente al usuario de MySQL.
       valor_host(str): el valor del host.
   Returns: aunque no tiene return, elimina una columna de la tabla especificada.
   """
    cnx = mysql.connector.connect(user=valor_user, password = valor_password,host = valor_host)

    mycursor = cnx.cursor()
    mycursor.execute("SHOW DATABASES")

    bases_de_datos = [database[0] for database in mycursor]

    if base_datos in bases_de_datos:
        resultado = conocer_columnas_tabla()
        print(f'ATENCION: Estas operando con la tabla {tabla}')
        columna = input(f'De estas columnas, {resultado} ¿Cuál quieres que sea eliminada?').lower()
        if columna in resultado:
            cnx = mysql.connector.connect(user = valor_user, password = valor_password, host = valor_host, database = base_datos)
            mycursor = cnx.cursor()
            query= f'ALTER TABLE {tabla} DROP COLUMN  {columna}'
            try: 
                mycursor.execute(query)
            except mysql.connector.Error as err:
                print(err)
                print("Error Code:", err.errno)
                print("SQLSTATE", err.sqlstate)
                print("Message", err.msg)
    else:

        eleccion_3 = input(f'No existe la base de datos {base_datos}, ¿Quieres crearla? (si/no)')
        if eleccion_3.lower() == 'si':
            crear_base_datos()

## Creamos la clase

In [44]:
class BBDD:
    
    def __init__(self,base_datos,tabla,valor_password,valor_user='root', valor_host='127.0.0.1',lista_columnas =[],indice=0):
        """
   Esta función es el constructor de la clase.
  Args: 
       tabla(str): el nombre de la tabla con la que se esta operando en ese momento.
       base_datos(str): el nombre de la base de datos con la que se esta operando en ese momento.
       valor_user(str): el nombre del usuario de MySQL.
       valor_password(str): contraseña correspondiente al usuario de MySQL.
       valor_host(str): el valor del host.
       lista columnas (lista): lista donde están los nombres de las columnas propuestas por el usuario.
       indice(int): el número de indice donde se encuentra el tipo de dato de la columa especificada.
   """
        self.base_datos = base_datos.lower()
        self.tabla = tabla.lower()
        self.valor_user = valor_user
        self.valor_password = valor_password
        self.valor_host = valor_host
        self.lista_columnas = lista_columnas
        self.indice = indice
    
    def crear_base_datos(self):
        """
   Esta función crea una base de datos de forma automatica.
  Args: 
       self: argumentos del constructor
   Returns: a pesar de no tener return, crea una base de datos en el servidor de MySQL proporcionado.
   """
        atencion = input(f'Vas a crear la base de datos {self.base_datos},¿Quieres cambiar el nombre? (si/no)').lower()
        if atencion == 'si':
            eleccion = input('¿Cómo quieres que se llame tu base de datos?')
            self.base_datos = eleccion.lower()


        cnx = mysql.connector.connect(user=self.valor_user, password = self.valor_password,host = self.valor_host)

        mycursor = cnx.cursor()

        try:

            mycursor.execute(f"CREATE DATABASE {self.base_datos}")
            print(f'Se ha creado la base de datos {self.base_datos}')
            print(f'Estas operando con la base de datos {self.base_datos}')
        except mysql.connector.Error as err:

            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg)

        opcion = input('¿Quieres crear otra base de datos?(si/no)')

        if opcion.lower() == 'si':
            nuevo_nombre = input('¿Cómo quieres que se llame la nueva base de datos?').lower()
            self.base_datos = nuevo_nombre
            self.crear_base_datos()

    def cambiar_base_datos(self):
        """
   Esta función cambia de base de datos.
  Args: 
       self: argumentos del constructor
   Returns: aunque no tiene return, cambia de base de datos.
   """
        cnx = mysql.connector.connect(user=self.valor_user, password = self.valor_password,host = self.valor_host)

        mycursor = cnx.cursor()
        mycursor.execute("SHOW DATABASES")

        bases_de_datos = [database[0] for database in mycursor]

        opcion = input(f'Estas operando actualmente con la base de datos {self.base_datos},¿Quieres continuar?(si/no)')

        if opcion.lower() == 'si':
            
            print(f'Las bases de datos disponibles son {bases_de_datos}')

            eleccion = input('Elige la base de datos con la quieres operar')

            if eleccion.lower() in bases_de_datos:

                self.base_datos = eleccion.lower()
                print(f'Estas operando con la base de datos {self.base_datos}')

            else:

                eleccion_2 = input('Esa eleccion no es correcta, ¿Quieres probar de nuevo? (si/no)')

                if eleccion_2.lower() == 'si':

                    self.cambiar_base_datos()

                else:

                    print(f'La base de datos con la que estás operando es {self.base_datos}')
        else:
            print(f'La base de datos con la que estás operando es {self.base_datos}')

    def crear_tabla(self):
        """
   Esta función crea una tabla de forma automatica.
  Args: 
       self: argumentos del constructor
   Returns: a pesar de no tener return, crea una tabla en una base de datos del servidor de MySQL proporcionado.
   """
        atencion = input(f'ATENCION: vas a operar con la base de datos {self.base_datos},¿Quieres continuar?(si/no)')
        cnx = mysql.connector.connect(user=self.valor_user, password = self.valor_password,host = self.valor_host)

        mycursor = cnx.cursor()
        mycursor.execute("SHOW DATABASES")

        bases_de_datos = [database[0] for database in mycursor]

        if self.base_datos in bases_de_datos:

            if atencion.lower() =='si':
                nombre_tabla = input('Indica el nombre de la tabla').lower()
                self.tabla = nombre_tabla
                nombre_columnas = input('Indica el nombre de las columnas separadas por comas')
                tipo_columnas = input('Indice el tipo de dato de cada columna separaas por comas')
                nulo = input('Indice si puede ser nulo o no')
                caracteres = input('¿Cuantos caracteres quieres que tengan los de tipo VARCHAR')
                nombre_columnas = nombre_columnas.split(',')
                tipo_columnas = tipo_columnas.split(',')
                nulo = nulo.split(',')
                nombre_columnas = [elemento.strip().lower() for elemento in nombre_columnas]
                tipo_columnas = [elemento.strip().upper() for elemento in tipo_columnas]
                nulo = [elemento.strip().lower() for elemento in nulo]
                if len(nombre_columnas) == len(tipo_columnas) == len(nulo):
                    query = f"""CREATE TABLE IF NOT EXISTS {self.tabla} ("""
                    for indice in range(len(nombre_columnas)):
                        if tipo_columnas[indice] in ['INT','VARCHAR','FLOAT']:
                            if indice == len(nombre_columnas) -1:
                                if tipo_columnas[indice] == 'VARCHAR':
                                    if nulo[indice].lower() == 'si':
                                        query = query + ' ' + nombre_columnas[indice] + ' ' + tipo_columnas[indice] + '(' + caracteres + ')' + ')'
                                    elif nulo[indice].lower() == 'no':
                                        query = query + ' ' + nombre_columnas[indice] + ' ' + tipo_columnas[indice] +'(' + caracteres + ')' + ' ' + 'NOT NULL' +')'
                                else:
                                    if nulo[indice].lower() == 'si':
                                        query = query + ' ' + nombre_columnas[indice] + ' ' + tipo_columnas[indice] + ')'
                                    elif nulo[indice].lower() == 'no':
                                        query = query + ' ' + nombre_columnas[indice] + ' ' + tipo_columnas[indice] + ' ' + 'NOT NULL' +')'
                            else:
                                if tipo_columnas[indice] == 'VARCHAR':
                                    if nulo[indice].lower() == 'si':
                                        query = query + ' ' + nombre_columnas[indice] + ' ' + tipo_columnas[indice] + '(' + caracteres + ')' + ','
                                    elif nulo[indice].lower() == 'no':
                                        query = query + ' ' + nombre_columnas[indice] + ' ' + tipo_columnas[indice] + '(' + caracteres + ')' + ' ' + 'NOT NULL' + ','
                                else:
                                    if nulo[indice].lower() == 'si':
                                        query = query + ' ' + nombre_columnas[indice] + ' ' + tipo_columnas[indice] + ','
                                    elif nulo[indice].lower() == 'no':
                                        query = query + ' ' + nombre_columnas[indice] + ' ' + tipo_columnas[indice] + ' ' + 'NOT NULL' + ','
                else:
                    print('No has especificado el mismo numero de elementos en las tres listas')
                cnx= mysql.connector.connect(user = self.valor_user, password = self.valor_password, host = self.valor_host, database = self.base_datos)
                mycursor = cnx.cursor()
                try: 
                    mycursor.execute(query)
                except mysql.connector.Error as err:
                    print(err)
                    print("Error Code:", err.errno)
                    print("SQLSTATE", err.sqlstate)
                    print("Message", err.msg)

            elif atencion.lower() == 'no':

                eleccion_2 = input('¿Quieres cambiar de base de datos (si/no)')

                if eleccion_2.lower() == 'si':
                    
                    self.cambiar_base_datos()
        else:
            eleccion_3 = input(f'No existe la base de datos {self.base_datos}, ¿Quieres crearla? (si/no)')
            if eleccion_3.lower() == 'si':
                self.crear_base_datos()


    def cambiar_tabla(self):
        """
   Esta función cambia de tabla.
  Args: 
       self: argumentos del constructor
   Returns: aunque no tiene return, cambia de tabla.
   """
        cnx = mysql.connector.connect(user=self.valor_user, password = self.valor_password,host = self.valor_host)

        mycursor = cnx.cursor()
        mycursor.execute("SHOW DATABASES")

        bases_de_datos = [database[0] for database in mycursor]

        if self.base_datos in bases_de_datos:

            cnx = mysql.connector.connect(database=self.base_datos,user=self.valor_user, password = self.valor_password,host = self.valor_host)

            mycursor = cnx.cursor()

            mycursor.execute("SHOW TABLES")

            conocer_tablas = [tablas[0] for tablas in mycursor]
            print(f'Las tablas disponibles son {conocer_tablas}')

            eleccion = input('Elige la tabla con la quieres operar')

            if eleccion.lower() in conocer_tablas:

                self.tabla = eleccion.lower()
                print(f'Estas operando con la base de datos {eleccion.lower()}')

            else:

                eleccion_2 = input('Esa eleccion no es correcta, ¿Quieres probar de nuevo? (si/no)')

                if eleccion_2.lower() == 'si':

                    self.cambiar_tabla()

                else:

                    print(f'La base de datos con la que estás operando es {self.tabla}')
        else:

            eleccion_3 = input(f'No existe la base de datos {self.base_datos}, ¿Quieres crearla? (si/no)')
            if eleccion_3.lower() == 'si':
                self.crear_base_datos()
    
    def conocer_tablas(self):
        """
   Esta función extrae las tablas de una base de datos.
  Args: 
       self: argumentos del constructor
   Returns: devuelve una lista con las tablas de la base de datos especificada.
   """
        cnx = mysql.connector.connect(user=self.valor_user, password = self.valor_password,host = self.valor_host)

        mycursor = cnx.cursor()
        mycursor.execute("SHOW DATABASES")

        bases_de_datos = [database[0] for database in mycursor]

        if self.base_datos in bases_de_datos:

            cnx = mysql.connector.connect(database=self.base_datos,user=self.valor_user, password = self.valor_password,host = self.valor_host)

            mycursor = cnx.cursor()

            mycursor.execute("SHOW TABLES")

            conocer_tablas = [tablas[0] for tablas in mycursor]
            return conocer_tablas
       
    
    def conocer_columnas_tabla(self):
        """
   Esta función extrae las columnas que hay una tabla concreta.
  Args: 
       self: argumentos del constructor
   Returns: devuelve una lista con todas las columnas que tiene la tabla proporcionada.
   """
        cnx = mysql.connector.connect(user = self.valor_user, password = self.valor_password, host = self.valor_host, database = self.base_datos)
        mycursor = cnx.cursor()
        try: 
            mycursor.execute(f'SHOW COLUMNS FROM {self.tabla}')
        except mysql.connector.Error as err:
            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg)
        resultado = [elemento[0] for elemento in mycursor]
        return resultado
    
    def tipo_dato(self):
        """
   Esta función extrae el tipo de dato de una columna.
  Args:
       self: argumentos del constructor.
   Returns: devuelve una lista con todas las columnas que tiene la tabla proporcionada.
        """
        cnx = mysql.connector.connect(database=self.base_datos,user=self.valor_user, password=self.valor_password,host=self.valor_host)
        mycursor = cnx.cursor()
        try: 
            mycursor.execute(f'SHOW COLUMNS FROM {self.tabla}')
        except mysql.connector.Error as err:
            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg)
        resultado = [elemento[1] for elemento in mycursor]
        return resultado[self.indice]
    
    def comprobar_columnas(self):
        """
   Esta función comprueba que las columnas proporcionadas están una tabla concreta.
  Args: 
       self: argumentos del constructor
   Returns: devuelve un booleano dependiendo si las columnas están presentes en la tabla o no.
   """
        resultado_columnas = self.conocer_columnas_tabla()

        for indice in range(len(self.lista_columnas)):

            if indice == len(self.lista_columnas)-1:

                if self.lista_columnas[indice] in resultado_columnas:
                    return True
            else:

                if self.lista_columnas[indice] not in resultado_columnas:
                    return False

    def clave_primaria(self):
        """
   Esta función establece la primary key de una tabla.
  Args: 
       self: argumentos del constructor
   Returns: aunque no tiene return, establece la PK de una tabla.
   """
        cnx = mysql.connector.connect(user=self.valor_user, password = self.valor_password,host = self.valor_host)

        mycursor = cnx.cursor()
        mycursor.execute("SHOW DATABASES")

        bases_de_datos = [database[0] for database in mycursor]

        if self.base_datos in bases_de_datos:
            resultado = self.conocer_columnas_tabla()
            print(f'ATENCION: Estas operando con la tabla{self.tabla}')
            columna = input(f'De estas columnas, {resultado} ¿Cuál quieres que sea PK?')
            nombre_cons = input('¿Como quieres que se llame la constraint?')
            if columna.lower() in resultado:
                cnx = mysql.connector.connect(user = self.valor_user, password = self.valor_password, host = self.valor_host, database = self.base_datos)
                mycursor = cnx.cursor()
                query= f'ALTER TABLE {self.tabla} ADD CONSTRAINT {nombre_cons} PRIMARY KEY ({columna});'
                try: 
                    mycursor.execute(query)
                except mysql.connector.Error as err:
                    print(err)
                    print("Error Code:", err.errno)
                    print("SQLSTATE", err.sqlstate)
                    print("Message", err.msg)
        else:

            eleccion_3 = input(f'No existe la base de datos {self.base_datos}, ¿Quieres crearla? (si/no)')
            if eleccion_3.lower() == 'si':
                self.crear_base_datos()
    
    def clave_foranea(self):
        """
   Esta función establace la clave foranea de una tabla.
  Args: 
       self: argumentos del constructor
   Returns: aunque no tiene return, mestablace la FK de la tabla especificada.
   """
        cnx = mysql.connector.connect(user=self.valor_user, password = self.valor_password,host = self.valor_host)

        mycursor = cnx.cursor()
        mycursor.execute("SHOW DATABASES")

        bases_de_datos = [database[0] for database in mycursor]

        if self.base_datos in bases_de_datos:
            resultado = self.conocer_columnas_tabla()
            print(f'ATENCION: La tabla que estas utilizando y donde especificaras la FK es {self.tabla}')
            columna_FK = input(f'De estas columnas, {resultado} ¿Cuál quieres que sea la FK?').lower()
            if columna_FK in resultado:
                tablas = self.conocer_tablas()
                eleccion_tabla_PK =input(f'De estas tablas, {tablas}¿Cuál tendrá relación con la columna?').lower()
                if eleccion_tabla_PK in tablas:
                    tabla_FK = self.tabla
                    self.tabla = eleccion_tabla_PK
                    resultado2 = self.conocer_columnas_tabla()
                    columna_PK = input(f'De estas columnas, {resultado2} ¿Cuál quieres que sea la que tenga relación con la FK?').lower()
                    if columna_PK in resultado2:
                        cnx = mysql.connector.connect(user = self.valor_user, password = self.valor_password, host = self.valor_host, database = self.base_datos)
                        mycursor = cnx.cursor()
                        query= f'ALTER TABLE {tabla_FK} ADD FOREIGN KEY ({columna_FK}) REFERENCES {self.tabla}  ({columna_PK})'
                        try: 
                            mycursor.execute(query)
                        except mysql.connector.Error as err:
                            print(err)
                            print("Error Code:", err.errno)
                            print("SQLSTATE", err.sqlstate)
                            print("Message", err.msg)
        else:

            eleccion_3 = input(f'No existe la base de datos {self.base_datos}, ¿Quieres crearla? (si/no)')
            if eleccion_3.lower() == 'si':
                self.crear_base_datos()

    def modificar_columna(self):
        """
   Esta función modifica el tipo de dato, si es nulo o si es autoincrementable una columna de una tabla.
  Args: 
       self: argumentos del constructor
   Returns: aunque no tiene return, modifica la columna especificada.
   """
        cnx = mysql.connector.connect(user=self.valor_user, password = self.valor_password,host = self.valor_host)

        mycursor = cnx.cursor()
        mycursor.execute("SHOW DATABASES")

        bases_de_datos = [database[0] for database in mycursor]

        if self.base_datos in bases_de_datos:
            resultado = self.conocer_columnas_tabla()
            print(f'ATENCION: Estas operando con la tabla{self.tabla}')
            columna = input(f'De estas columnas, {resultado} ¿Cuál quieres que sea modificada?').lower()
            tipo_dato = input('¿Como quieres que sea el tipo de dato?').upper()
            auto = input('¿Quiers que sea auto-incrementable?')
            if tipo_dato == 'VARCHAR':
                caracteres = input('Especifica el numero de caracteres')
            nulo = input('¿Quieres que pueda ser nulo').lower()
            if columna in resultado:
                cnx = mysql.connector.connect(user = self.valor_user, password = self.valor_password, host = self.valor_host, database = self.base_datos)
                mycursor = cnx.cursor()
                query= f'ALTER TABLE {self.tabla} MODIFY COLUMN  {columna}'
                if tipo_dato in ['INT','VARCHAR','FLOAT']:
                    if tipo_dato == 'VARCHAR':
                        if nulo == 'si':
                            query = query + ' ' + tipo_dato + '('+ caracteres + ')'
                        elif nulo == 'no':
                            query = query + ' ' + tipo_dato + '('+ caracteres + ')' + ' '+'NOT NULL'
                    else:
                        if auto.lower() == 'si':
                            if nulo == 'si':
                                query = query + ' ' + tipo_dato
                            elif nulo == 'no':
                                query = query + ' ' + tipo_dato + ' ' + 'NOT NULL' + ' ' + 'AUTO_INCREMENT'
                        else:
                            if nulo == 'si':
                                query = query + ' ' + tipo_dato 
                            elif nulo == 'no':
                                query = query + ' ' + tipo_dato + ' ' + 'NOT NULL'
                try: 
                    mycursor.execute(query)
                except mysql.connector.Error as err:
                    print(err)
                    print("Error Code:", err.errno)
                    print("SQLSTATE", err.sqlstate)
                    print("Message", err.msg)
        else:

            eleccion_3 = input(f'No existe la base de datos {self.base_datos}, ¿Quieres crearla? (si/no)')
            if eleccion_3.lower() == 'si':
                self.crear_base_datos()
    
    def eliminar_columna(self):
        """
   Esta función elimina una columna.
  Args: 
       self: argumentos del constructor
   Returns: aunque no tiene return, elimina una columna de la tabla especificada.
   """
        cnx = mysql.connector.connect(user=self.valor_user, password = self.valor_password,host = self.valor_host)

        mycursor = cnx.cursor()
        mycursor.execute("SHOW DATABASES")

        bases_de_datos = [database[0] for database in mycursor]

        if self.base_datos in bases_de_datos:
            resultado = self.conocer_columnas_tabla()
            print(f'ATENCION: Estas operando con la tabla {self.tabla}')
            columna = input(f'De estas columnas, {resultado} ¿Cuál quieres que sea eliminada?').lower()
            if columna in resultado:
                cnx = mysql.connector.connect(user = self.valor_user, password = self.valor_password, host = self.valor_host, database = self.base_datos)
                mycursor = cnx.cursor()
                query= f'ALTER TABLE {self.tabla} DROP COLUMN  {columna}'
                try: 
                    mycursor.execute(query)
                except mysql.connector.Error as err:
                    print(err)
                    print("Error Code:", err.errno)
                    print("SQLSTATE", err.sqlstate)
                    print("Message", err.msg)
        else:

            eleccion_3 = input(f'No existe la base de datos {self.base_datos}, ¿Quieres crearla? (si/no)')
            if eleccion_3.lower() == 'si':
                self.crear_base_datos()
                
    def insertar_columna(self):
        """
   Esta función introduce una nueva columna en una tabla.
  Args: 
       self: argumentos del constructor
   Returns: aunque no tiene return, inserta una nueva columna en la tabla especificada.
   """
        print(f'ATENCION: Estas operando con la tabla{self.tabla}')
        columna = input(f'Escribe el nombre de la nueva columna').lower()
        tipo_dato = input('¿Como quieres que sea el tipo de dato?').upper()
        if tipo_dato == 'VARCHAR':
            caracteres = input('Especifica el numero de caracteres')
        nulo = input('¿Quieres que pueda ser nulo').lower()

        cnx = mysql.connector.connect(user=self.valor_user, password = self.valor_password,host = self.valor_host)

        mycursor = cnx.cursor()
        mycursor.execute("SHOW DATABASES")

        bases_de_datos = [database[0] for database in mycursor]

        if self.base_datos in bases_de_datos:
    
            cnx = mysql.connector.connect(user = self.valor_user, password = self.valor_password, host = self.valor_host, database = self.base_datos)
            mycursor = cnx.cursor()
            query= f'ALTER TABLE {self.tabla} ADD COLUMN  {columna}'
            if tipo_dato in ['INT','VARCHAR','FLOAT']:
                if tipo_dato == 'VARCHAR':
                    if nulo == 'si':
                        query = query + ' ' + tipo_dato + '('+ caracteres + ')'
                    elif nulo == 'no':
                        query = query + ' ' + tipo_dato + '('+ caracteres + ')' + ' ' + 'NOT NULL'
                else:
                    if nulo == 'si':
                        query = query + ' ' + tipo_dato
                    elif nulo == 'no':
                        query = query + ' ' + tipo_dato + ' ' + 'NOT NULL'
            try: 
                mycursor.execute(query)
            except mysql.connector.Error as err:
                print(err)
                print("Error Code:", err.errno)
                print("SQLSTATE", err.sqlstate)
                print("Message", err.msg)
        else:

            eleccion_3 = input(f'No existe la base de datos {self.base_datos}, ¿Quieres crearla? (si/no)')
            if eleccion_3.lower() == 'si':
                self.crear_base_datos()

    def insertar_datos(self,df):
        """
   Esta función inserta datos en una tabla.
  Args: 
       df(dataframe): el dataframe que contiene los datos que queremos insertar en la tabla.
       self: argumentos del constructor
   Returns: aunque no tiene return, inserta todos los datos en la tabla especificada.
   """
        cnx = mysql.connector.connect(user=self.valor_user, password = self.valor_password,host = self.valor_host)

        mycursor = cnx.cursor()
        mycursor.execute("SHOW DATABASES")

        bases_de_datos = [database[0] for database in mycursor]

        if self.base_datos in bases_de_datos:

            resultado = self.conocer_columnas_tabla()
            print(f'ATENCION: Estas operando con la tabla{self.tabla}')
            eleccion = input(f'De estas columnas {resultado}, ¿En cuales vas a introducir datos?')
            eleccion = eleccion.split(',')
            self.lista_columnas = eleccion
            
            if self.comprobar_columnas():
                num_valores ='(%s'

                for i in range(len(eleccion)-1):
                    num_valores = num_valores +',%s'
                num_valores = num_valores+')'
                
                codigo_sql=f'INSERT INTO {self.tabla} {tuple(eleccion)} VALUES {num_valores}'
                codigo_sql=codigo_sql.replace("'","")
                lista = []
                for indice, fila in df.iterrows():
                    lista=[]
                    for i in df.loc[indice,:].values:
                        lista.append(i)
                    valores = tuple(lista)
                    cnx = mysql.connector.connect(user = self.valor_user, password = self.valor_password, host = self.valor_host, database = self.base_datos)
                    mycursor = cnx.cursor()
                                    
                    try: 
                        mycursor.execute(codigo_sql, valores)
                        cnx.commit()
                    except mysql.connector.Error as err:
                        print(err)
                        print("Error Code:", err.errno)
                        print("SQLSTATE", err.sqlstate)
                        print("Message", err.msg)
            else:
                eleccion_2 = input('Las columnas introducidas no corresponde con la tabla, ¿Quieres volverlo a intentar? (si/no)').lower()
                if eleccion_2 == 'si':
                    self.insertar_datos()
        else:
            eleccion_3 = input(f'No existe la base de datos {self.base_datos}, ¿Quieres crearla? (si/no)')
            if eleccion_3.lower() == 'si':
                self.crear_base_datos()   

    def eliminar_registro(self):
        """
   Esta función elimina todos los registros de la tabla que cumple una cierta condición.
  Args: 
       self: argumentos del constructor
   Returns: aunque no tiene return, elimina los registros especificados.
   """
        cnx = mysql.connector.connect(user=self.valor_user, password = self.valor_password,host = self.valor_host)

        mycursor = cnx.cursor()
        mycursor.execute("SHOW DATABASES")

        bases_de_datos = [database[0] for database in mycursor]

        if self.base_datos in bases_de_datos:
            resultado = self.conocer_columnas_tabla()
            print(f'ATENCION: Estas operando con la tabla{self.tabla}')
            columna = input(f'¿Qué columna tomamos como referencia para borrar el registro?{resultado}').lower()
            if columna in resultado:
                cnx = mysql.connector.connect(user = self.valor_user, password = self.valor_password, host = self.valor_host, database = self.base_datos)
                mycursor = cnx.cursor()
                query = f'SELECT DISTINCT {columna} FROM {self.tabla}'
                query=query.replace("'","")
                try: 
                    mycursor.execute(query)
                except mysql.connector.Error as err:
                    print(err)
                    print("Error Code:", err.errno)
                    print("SQLSTATE", err.sqlstate)
                    print("Message", err.msg)

                elementos_unicos = [elemento[0] for elemento in mycursor]
                eleccion_elemento = input(f'Entre los registros disponibles: {elementos_unicos}, ¿Cuál quieres borrar?').lower().capitalize()
                self.indice = resultado.index(columna)
                tipo_columna = self.tipo_dato()
                if tipo_columna in ['int']:
                    eleccion_elemento = int(eleccion_elemento)
                    if eleccion_elemento in elementos_unicos:
                            
                            codigo_sql=f"DELETE FROM {self.tabla} WHERE {columna} = {eleccion_elemento}" 
                            cnx = mysql.connector.connect(user = self.valor_user, password = self.valor_password, host = self.valor_host, database = self.base_datos)
                            mycursor = cnx.cursor()
                                                
                            try: 
                                mycursor.execute(codigo_sql)
                                cnx.commit()
                                print('Se ha eliminado correctamente todos los registros con esa condición en la base de datos.')
                            except mysql.connector.Error as err:
                                print(err)
                                print("Error Code:", err.errno)
                                print("SQLSTATE", err.sqlstate)
                                print("Message", err.msg)
                else:
                    if eleccion_elemento in elementos_unicos:
                        codigo_sql=f"DELETE FROM {self.tabla} WHERE {columna} = '{eleccion_elemento}'" 
                        cnx = mysql.connector.connect(user = self.valor_user, password = self.valor_password, host = self.valor_host, database = self.base_datos)
                        mycursor = cnx.cursor()
                                            
                        try: 
                            mycursor.execute(codigo_sql)
                            cnx.commit()
                            print('Se ha eliminado correctamente todos los registros con esa condición en la base de datos.')
                        except mysql.connector.Error as err:
                            print(err)
                            print("Error Code:", err.errno)
                            print("SQLSTATE", err.sqlstate)
                            print("Message", err.msg)
            else:
                eleccion_3 = input(f'No existe ningún registro con esas caracteristicas, ¿Quieres probar de nuevo? (si/no)')
                if eleccion_3.lower() == 'si':
                    self.eliminar_registro()
        else:
            eleccion_3 = input(f'No existe la base de datos {self.base_datos}, ¿Quieres crearla? (si/no)')
            if eleccion_3.lower() == 'si':
                self.crear_base_datos() 
    
    def consultar_registro(self):
        """
   Esta función consulta todos los registros de la tabla que cumple una cierta condición.
  Args: 
       self: argumentos del constructor
   Returns: aunque no tiene return, consulta los registros especificados.
   """
        cnx = mysql.connector.connect(user=self.valor_user, password = self.valor_password,host = self.valor_host)

        mycursor = cnx.cursor()
        mycursor.execute("SHOW DATABASES")

        bases_de_datos = [database[0] for database in mycursor]

        if self.base_datos in bases_de_datos:
            resultado = self.conocer_columnas_tabla()
            print(f'ATENCION: Estas operando con la tabla{self.tabla}')
            columna = input(f'¿Qué columna tomamos como referencia para consultar el registro?{resultado}').lower()
            if columna in resultado:
                cnx = mysql.connector.connect(user = self.valor_user, password = self.valor_password, host = self.valor_host, database = self.base_datos)
                mycursor = cnx.cursor()
                query = f'SELECT DISTINCT {columna} FROM {self.tabla}'
                query=query.replace("'","")
                try: 
                    mycursor.execute(query)
                except mysql.connector.Error as err:
                    print(err)
                    print("Error Code:", err.errno)
                    print("SQLSTATE", err.sqlstate)
                    print("Message", err.msg)

                elementos_unicos = [elemento[0] for elemento in mycursor]
                eleccion_elemento = input(f'Entre los registros disponibles: {elementos_unicos}, ¿Cuál quieres consultar?').lower().capitalize()
                self.indice = resultado.index(columna)
                tipo_columna = self.tipo_dato()
                if tipo_columna in ['int']:
                    eleccion_elemento = int(eleccion_elemento)
                    if eleccion_elemento in elementos_unicos:
                            
                            codigo_sql=f"SELECT * FROM {self.tabla} WHERE {columna} = {eleccion_elemento}" 
                            cnx = mysql.connector.connect(user = self.valor_user, password = self.valor_password, host = self.valor_host, database = self.base_datos)
                            mycursor = cnx.cursor()
                                                
                            try: 
                                mycursor.execute(codigo_sql)
                                registros = [elemento for elemento in mycursor]
                                return registros
                            except mysql.connector.Error as err:
                                print(err)
                                print("Error Code:", err.errno)
                                print("SQLSTATE", err.sqlstate)
                                print("Message", err.msg)
                else:
                    if eleccion_elemento in elementos_unicos:
                        codigo_sql=f"SELECT * FROM {self.tabla} WHERE {columna} = '{eleccion_elemento}'" 
                        cnx = mysql.connector.connect(user = self.valor_user, password = self.valor_password, host = self.valor_host, database = self.base_datos)
                        mycursor = cnx.cursor()
                                            
                        try: 
                            mycursor.execute(codigo_sql)
                            registros = [elemento for elemento in mycursor]
                            return registros
                        except mysql.connector.Error as err:
                            print(err)
                            print("Error Code:", err.errno)
                            print("SQLSTATE", err.sqlstate)
                            print("Message", err.msg)
            else:
                eleccion_3 = input(f'No existe ningún registro con esas caracteristicas, ¿Quieres probar de nuevo? (si/no)')
                if eleccion_3.lower() == 'si':
                    self.consultar_registro()
        else:
            eleccion_3 = input(f'No existe la base de datos {self.base_datos}, ¿Quieres crearla? (si/no)')
            if eleccion_3.lower() == 'si':
                self.crear_base_datos()

            

## Pruebas de la clase

In [47]:
prueba = BBDD('base_datos_prueba_1','tabla_1','AlumnaAdalab')

In [35]:
prueba.crear_base_datos()

Se ha creado la base de datos base_datos_prueba_1
Estas operando con la base de datos base_datos_prueba_1


In [81]:
prueba.cambiar_base_datos()

Las bases de datos disponibles son ['base_datos_prueba', 'BD_Alumna', 'BD_pruebas', 'creacion_tienda', 'Ejercicios', 'energia', 'information_schema', 'leccion-1-sql', 'leccion-10-sql', 'leccion-11-sql', 'leccion-12-sql', 'leccion-13-sql', 'leccion-14-sql', 'leccion-2-sql', 'leccion-3-sql', 'leccion-4-sql', 'leccion-5-sql', 'leccion-6-sql', 'leccion-7-sql', 'leccion-8-sql', 'leccion-9-sql', 'musica_promo_e', 'mysql', 'northwind', 'performance_schema', 'pr1e4_adalab', 'proyecto_tienda', 'prueba_pr1e4_adalab', 'pueblo', 'sakila', 'Sports_Analytics', 'sys', 'tienda', 'tienda_2', 'tienda_zapatillas', 'world']
Estas operando con la base de datos base_datos_prueba


In [40]:
prueba.crear_tabla()

In [41]:
prueba.clave_primaria()

ATENCION: Estas operando con la tablatabla_2


In [42]:
prueba.modificar_columna()

ATENCION: Estas operando con la tablatabla_2


In [43]:
prueba.cambiar_tabla()

Las tablas disponibles son ['tabla_1', 'tabla_2']


Estas operando con la base de datos tabla_1


In [106]:
prueba.insertar_columna()

ATENCION: Estas operando con la tablatabla2


In [104]:
prueba.eliminar_columna()

ATENCION: Estas operando con la tabla tabla1


In [48]:
prueba.clave_foranea()

ATENCION: La tabla que estas utilizando y donde especificaras la FK es tabla_1


In [131]:
prueba.insertar_datos(df_prueba)

ATENCION: Estas operando con la tablatabla1


In [11]:
prueba.eliminar_registro()

ATENCION: Estas operando con la tablatabla1
Se ha eliminado correctamente todos los registros con esa condición en la base de datos.


In [14]:
prueba.consultar_registro()

ATENCION: Estas operando con la tablatabla1


[(7, 'Alli', 'Alligator', 'Female')]